In [1]:
import sys
import os

# Adjust the path to point to your src directory
sys.path.insert(0, os.path.abspath('../../src'))

import xarray as xr

from icpoes.unit_conv import compare_to_standards
from icpoes.utils import find_common_elements
from icpoes.utils import average_wavelengths
from icpoes.errors import append_stats_to_dataarray
from icpoes.utils import select_by_fuzzy_sample
from icpoes.utils import drop_labels_by_suffix
from icpoes.check_stds import check_standards
from icpoes.errors import append_percentage_difference
from icpoes.errors import append_error_estimate
from icpoes.errors import add_error_variable

In [3]:
file_list = ['../../data/sukinda.nc', '../../data/soils.nc', '../../data/deccan.nc']

for file_name in file_list:
    ds = xr.open_dataset(file_name)
    # Get good and possible wavelengths
    good_wavelengths, possible_wavelengths = compare_to_standards(ds)
    # Find wavelengths common to both check standards that are within the 5% limit
    sure_wavelengths = find_common_elements(good_wavelengths)
    # Get a set of all wavelengths that are good
    combined = {item for sublist in good_wavelengths.values() for item in sublist}

    # Line to force calculation of all species regardless
    combined = ds.coords['species'].values
    
    
    da = average_wavelengths(ds, combined)

    for chk_std in check_standards().keys():
        certified_values = check_standards()[chk_std][0]
        subset_labels = select_by_fuzzy_sample(ds, chk_std)
        subset_labels = subset_labels.coords['sample_name'].values.tolist()
        label_prefix = chk_std
        # Append statistics
        da = append_stats_to_dataarray(da, subset_labels, label_prefix)
        # Calculate precentatge difference from certified values
        da = append_percentage_difference(da, certified_values, chk_std)
        # Append the errors calculated from each check standard to the DataArray
        da = append_error_estimate(da, chk_std)

    # Compare check standard errors to the list of wavelengths
    # Pick check standard errors that correspond to the wavelengths used to generated the average concentration
    # If both check standards used, pick the largest error
    # Add error as a variable and return as dataset
    ds = add_error_variable(good_wavelengths, da)
    # Drop sample_names that were added for processing the check standards before saving
    ds = drop_labels_by_suffix(ds, ['diff', 'error', '2sd'], 'sample_name')

    ds.to_netcdf(f'{file_name.split(".nc")[0]}_errors.nc')

*** SLRS-6 ***
*** Al ***
SLRS-6 certified value: 0.00125 ± 8.2e-05 mM
Al 396.152 Ax: 0.00137 ± 0.000116 mM, 9.41%
Al 396.152 R: 0.00139 ± 0.000139 mM, 11.54%
*** Ba ***
SLRS-6 certified value: 0.000104 ± 3.5e-06 mM
Ba 455.403 Ax: 0.000106 ± 4.18e-06 mM, 1.95%
Ba 455.403 R: 0.000107 ± 4.75e-06 mM, 2.65%
*** Ca ***
SLRS-6 certified value: 0.218 ± 0.005 mM
Ca 315.887 R: 0.236 ± 0.0186 mM, 8.25%
Ca 317.933 R: 0.239 ± 0.0191 mM, 9.75%
Ca 396.847 Ax: 0.214 ± 0.00845 mM, -1.80%
Ca 396.847 R: 0.223 ± 0.00973 mM, 2.38%
Ca 422.673 R: 0.221 ± 0.00933 mM, 1.35%
*** Fe ***
SLRS-6 certified value: 0.00151 ± 6.4e-05 mM
Fe 238.204 R: 0.00152 ± 0.000123 mM, 0.59%
Fe 259.940 Ax: 0.00143 ± 7.41e-05 mM, -5.39%
Fe 259.940 R: 0.00145 ± 0.000123 mM, -3.71%
*** K ***
SLRS-6 certified value: 0.0167 ± 0.0014 mM
K 766.491 Ax: 0.0172 ± 0.000486 mM, 3.15%
K 766.491 R: 0.0165 ± 0.00122 mM, -1.10%
K 769.897 Ax: 0.0172 ± 0.000648 mM, 3.25%
*** Mg ***
SLRS-6 certified value: 0.0877 ± 0.0024 mM
Mg 279.078 R: 0.0898 ± 

/Users/angus/miniconda3/envs/topepan/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/angus/miniconda3/envs/topepan/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/angus/miniconda3/envs/topepan/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/angus/miniconda3/envs/topepan/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/angus/miniconda3/envs/topepan/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var =

K 766.491 Ax: 0.00305 ± 8.66e-05 mM, 19.25%
K 766.491 R: nan ± nan mM, nan%
K 769.897 Ax: 0.00362 ± 0.000115 mM, 41.39%
*** Mg ***
SPS-SW2 10% certified value: 0.00823 ± 4.11e-05 mM
Mg 279.078 R: 0.00846 ± 0.000817 mM, 2.74%
Mg 279.553 R: 0.00788 ± 0.000291 mM, -4.29%
Mg 279.800 R: 0.00807 ± 0.00078 mM, -1.96%
Mg 280.270 Ax: nan ± nan mM, nan%
Mg 280.270 R: 0.00795 ± 0.000307 mM, -3.39%
Mg 285.213 Ax: 0.00805 ± 0.000316 mM, -2.16%
Mg 285.213 R: 0.00794 ± 0.0003 mM, -3.49%
*** Na ***
SPS-SW2 10% certified value: 0.0435 ± 0.000217 mM
Na 588.995 Ax: 0.046 ± 0.00206 mM, 5.70%
Na 588.995 R: 0.0499 ± 0.00227 mM, 14.64%
Na 589.592 R: 0.0505 ± 0.00241 mM, 15.98%
*** Si ***
SPS-SW2 10% certified value: 0.0178 ± 0.000107 mM
Si 250.690 R: 0.0198 ± 0.00087 mM, 11.20%
Si 251.611 Ax: 0.0199 ± 0.00114 mM, 11.55%
Si 251.611 R: 0.0195 ± 0.000643 mM, 9.60%
Si 288.158 Ax: 0.02 ± 0.000945 mM, 12.35%
Si 288.158 R: 0.0198 ± 0.0012 mM, 11.20%
*** Li ***
SPS-SW2 10% does not have a certified value for Li
*** 

/Users/angus/miniconda3/envs/topepan/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/angus/miniconda3/envs/topepan/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
